<h2 style="text-align: center;">Data Mining & Grundlagen Maschinelles Lernen 1</h2>
<h3 style="text-align: center;">Wintersemester 2023/24</h3>
<h4 style="text-align: center;">Semesterprojekt: Umsatzvorhersage</h4>


# Business Understanding

Im Rahmen des Semesterprojekts für das Modul "Data Mining & Grundlagen Maschinelles Lernen 1" widmen wir uns einer praxisnahen Aufgabe – der Umsatzvorhersage für eine große Supermarktkette in Deutschland. Unser Ziel ist es, ein zuverlässiges Vorhersagemodell zu entwickeln, das auf Basis historischer Daten und verschiedener Einflussfaktoren den erwarteten Tagesumsatz jeder Filiale prognostiziert.
Eine genaue Vorhersage ermöglicht es den Filialleitungen, die Anzahl der Mitarbeiterinnen und Mitarbeiter optimal zu planen und so die Kundenzufriedenheit zu maximieren, während gleichzeitig die Personalkosten minimiert werden.

# Data Understanding

In [271]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from math import *
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, accuracy_score, r2_score
from sklearn.tree import DecisionTreeRegressor
import time
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
from sklearn.inspection import permutation_importance
import plotly.express as px
import matplotlib as plt
import seaborn as sns

In [53]:
# Schritt 1: Laden der Datensätze
train_df = pd.read_csv('../data/raw/dmml1_train.csv')
store = pd.read_csv('../data/raw/dmml1_stores.csv')

In [276]:
train_df.describe()

,Store ID,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday
count,246903.000000,246903.000000,246903.000000,246903.000000,246903.000000,246903.000000,246903.000000
mean,150.568879,3.996898,5844.491894,639.519524,0.830354,0.380490,0.178763
std,86.651448,1.998008,4007.013717,472.495279,0.375322,0.485508,0.383154
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,76.000000,2.000000,3745.000000,405.000000,1.000000,0.000000,0.000000
50%,150.000000,4.000000,5719.000000,617.000000,1.000000,0.000000,0.000000
75%,226.000000,6.000000,7884.000000,833.000000,1.000000,1.000000,0.000000
max,300.000000,7.000000,38025.000000,7388.000000,1.000000,1.000000,1.000000


In [277]:
store.describe()

,Store ID,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear
count,300.000000,298.000000,190.000000,190.000000,300.000000,141.000000,141.000000
mean,150.500000,5368.590604,7.189474,2007.794737,0.470000,22.886525,2011.978723
std,86.746758,7694.152795,3.236023,9.329079,0.499933,14.235816,1.627742
min,1.000000,30.000000,1.000000,1900.000000,0.000000,1.000000,2009.000000
25%,75.750000,682.500000,4.000000,2006.000000,0.000000,10.000000,2011.000000
50%,150.500000,2270.000000,8.000000,2009.000000,0.000000,22.000000,2012.000000
75%,225.250000,7107.500000,10.000000,2012.000000,1.000000,37.000000,2013.000000
max,300.000000,58260.000000,12.000000,2015.000000,1.000000,50.000000,2015.000000


In [278]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246903 entries, 0 to 246902
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Store ID       246903 non-null  int64 
 1   DayOfWeek      246903 non-null  int64 
 2   Date           246903 non-null  object
 3   Sales          246903 non-null  int64 
 4   Customers      246903 non-null  int64 
 5   Open           246903 non-null  int64 
 6   Promo          246903 non-null  int64 
 7   StateHoliday   246903 non-null  object
 8   SchoolHoliday  246903 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 17.0+ MB


In [279]:
store.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Store ID                   300 non-null    int64  
 1   StoreType                  300 non-null    object 
 2   Assortment                 300 non-null    object 
 3   CompetitionDistance        298 non-null    float64
 4   CompetitionOpenSinceMonth  190 non-null    float64
 5   CompetitionOpenSinceYear   190 non-null    float64
 6   Promo2                     300 non-null    int64  
 7   Promo2SinceWeek            141 non-null    float64
 8   Promo2SinceYear            141 non-null    float64
 9   PromoInterval              141 non-null    object 
dtypes: float64(5), int64(2), object(3)
memory usage: 23.6+ KB


In [280]:
train_df.isna().sum()

Store ID         0
DayOfWeek        0
Date             0
Sales            0
Customers        0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

In [281]:
store.isna().sum()

Store ID                       0
StoreType                      0
Assortment                     0
CompetitionDistance            2
CompetitionOpenSinceMonth    110
CompetitionOpenSinceYear     110
Promo2                         0
Promo2SinceWeek              159
Promo2SinceYear              159
PromoInterval                159
dtype: int64

# Data Preparation

In [503]:
train_df = pd.read_csv('../data/raw/dmml1_train.csv')
store = pd.read_csv('../data/raw/dmml1_stores.csv')

In [504]:
train_df['StateHoliday'] = train_df['StateHoliday'].map({'0': "Kein Feiertag", 'a': "Feiertag", 'b': "Ostern", 'c': "Weihnachten"})
store['StoreType'] = store['StoreType'].map({'a': "StoreType A", 'b': "StoreTypee B", 'c': "StoreType C", 'd': "StoreType D"})
store['Assortment'] = store['Assortment'].map({'a': "Basic", 'b': "Extra", 'c': "Extended"})

In [505]:
store["CompetitionDistance"].replace(np.nan, store["CompetitionDistance"].mean(), inplace=True)

train_df['Date'] = pd.to_datetime(train_df['Date'])
train_df['Year'] = train_df['Date'].dt.year
train_df['Month'] = train_df['Date'].dt.month
train_df['Day'] = train_df['Date'].dt.day
train_df['WeekOfYear'] = train_df['Date'].dt.isocalendar().week

In [506]:
train_df['Season'] = train_df['Month'].apply(lambda month: (month%12 // 3 + 1))

In [507]:
train_df['Temp'] = np.where(train_df['StateHoliday'] != 'Kein Feiertag', 1, 0)

# Erstellen Sie die Spalte "TagVorFeiertag", die 1 ist, wenn der nächste Tag ein Feiertag ist, und 0 sonst
train_df['TagVorFeiertag'] = train_df['Temp'].shift(-1).fillna(0)

# Erstellen Sie die Spalte "TagnachFeiertag", die 1 ist, wenn der vorherige Tag ein Feiertag war, und 0 sonst
train_df['TagnachFeiertag'] = train_df['Temp'].shift(1).fillna(0)
train_df.drop('Temp', axis=1, inplace=True)

In [508]:
train_df = pd.merge(train_df,store,how="inner",on="Store ID")
train_df.fillna(0,inplace=True)

# Exploratory Data Analysis

## Durchschnittliche Verkäufe

In [114]:
average_sales_per_day = train_df.groupby('Date')['Sales'].mean().reset_index()

fig = px.line(average_sales_per_day, y='Sales', x='Date', 
        labels={'Date': 'Kalendertag', 'Sales': 'Durchschnittlicher Umsatz'},
        title='Durchschnittlicher Umsatz pro Tag')
fig.show()

## Durchschnittliche Verkäufe pro Monat

In [319]:
average_sales_per_month = train_df.groupby('Month')['Sales'].mean().reset_index()

fig = px.bar(average_sales_per_month, x='Month', y='Sales',
             labels={'Month': 'Monat', 'Sales': 'Durchschnittlicher Umsatz'},
             title='Durchschnittlicher Umsatz pro Monat')
fig.update_xaxes(tickmode='array', tickvals=list(range(1, 13)),
                 ticktext=['Januar', 'Februar', 'März', 'April', 'Mai', 'Juni', 'Juli', 'August', 'September', 'Oktober', 'November', 'Dezember'])
fig.show()

## Durchschnittliche Verkäufe pro Woche

In [112]:
average_sales_per_week = train_df.groupby('WeekOfYear')['Sales'].mean().reset_index()

fig = px.line(average_sales_per_week, x='WeekOfYear', y='Sales',
              labels={'WeekOfYear': 'Kalenderwoche', 'Sales': 'Durchschnittlicher Umsatz'},
              title='Durchschnittlicher Umsatz pro Woche')
fig.update_xaxes(tickmode='array', tickvals=list(range(1, 54)))
fig.show()

- Der Umsatz variiert über das Jahr, mit einigen Spitzen und Tälern, was auf saisonale Einflüsse oder besondere Ereignisse hindeuten könnte. 
- Es gibt bestimmte Wochen, in denen der Umsatz deutlich höher ist als im Durchschnitt, was auf spezielle Verkaufsaktionen, Feiertage oder saisonale Ereignisse hinweisen könnte.

## Durchschnittliche Verkäufe pro Wochentag

In [296]:
average_sales_per_day = train_df.groupby('DayOfWeek')['Sales'].mean().reset_index()

fig = px.bar(average_sales_per_day, x='DayOfWeek', y='Sales',
             labels={'DayOfWeek': 'Wochentag', 'Sales': 'Durchschnittliche Verkäufe'},
             title='Durchschnittliche Verkäufe pro Wochentag')

fig.update_xaxes(tickmode='array', tickvals=[1, 2, 3, 4, 5, 6, 7],
                ticktext=['Montag', 'Dienstag', 'Mittwoch', 'Donnerstag', 'Freitag', 'Samstag', 'Sonntag'])
fig.show()

- Die Analyse der durchschnittlichen täglichen Verkäufe zeigt, dass der Beginn der Woche, insbesondere der Montag, für die Supermarktkette am umsatzstärksten ist, während die Verkaufszahlen gegen das Wochenende hin tendenziell abnehmen. 
- Auffallend ist der deutliche Einbruch der Verkaufszahlen am Sonntag, was darauf hindeutet, dass die meisten Filialen entweder geschlossen sind oder nur sehr begrenzt operieren.

In [130]:
average_sales_per_month = train_df.groupby('Month')['Sales'].mean().reset_index()

fig = px.bar(average_sales_per_month, x='Month', y='Sales',
             labels={'Month': 'Monat', 'Sales': 'Durchschnittlicher Umsatz'},
             title='Durchschnittlicher Umsatz pro Monat')
fig.update_xaxes(tickmode='array', tickvals=list(range(1, 13)),
                 ticktext=['Jan', 'Feb', 'Mär', 'Apr', 'Mai', 'Jun', 'Jul', 'Aug', 'Sep', 'Okt', 'Nov', 'Dez'])
fig.show()


- Der Umsatz variiert von Monat zu Monat, mit einigen Monaten, die höhere Durchschnittswerte aufweisen als andere.
- Es gibt keine extremen Spitzen oder Täler, aber es gibt eine sichtbare Variation im Bereich von etwa 5.570 € bis 6.883 € pro Monat im Durchschnitt.

# Aufgabe 1: Untersuchen Sie die Daten. Beantworten Sie dabei folgende Fragen bzw. Aufgaben

### 1. Wie viele Datenpunkte gibt es pro Store?

In [198]:
store_counts = train_df.groupby('Store ID').size().reset_index()
store_counts.columns = ['Store ID', 'Data Points']
unique_values = store_counts['Data Points'].value_counts().sort_index()

In [203]:
for i, v in unique_values.items():
    print(f"{v} Stores mit {i} Datenpunkten")


44 Stores mit 666 Datenpunkten
1 Stores mit 849 Datenpunkten
255 Stores mit 850 Datenpunkten


### 2. Visualisieren Sie die Verteilung der Verkaufszahlen für einige zufällig ausgewählte Stores und beschreiben Sie typische Muster, die Sie erkennen.

In [258]:
random_stores = np.random.choice(train_df['Store ID'].unique(), 5)
selected_stores_df = train_df[train_df['Store ID'].isin(random_stores)]

fig = px.histogram(selected_stores_df, x="Sales", color="Store ID", marginal="box", 
                   barmode="overlay", 
                   title="Verkaufszahlenverteilung für zufällig ausgewählte Stores",
                   height=600)

fig.update_xaxes(title_text='Verkaufszahlen')
fig.update_yaxes(title_text='Häufigkeit')
fig.show()

`Rechtsschiefe Verteilung`: Die meisten Filialen zeigen eine rechtsschiefe Verteilung der Verkaufszahlen. Das bedeutet, dass ein Großteil der Verkaufstage Verkaufszahlen im niedrigeren bis mittleren Bereich aufweist, während sehr hohe Verkaufszahlen seltener sind.

`Spitzen bei niedrigen Verkaufszahlen`: Es gibt Spitzen im Bereich sehr niedriger Verkaufszahlen. Dies könnte auf Tage hindeuten, an denen die Filialen geschlossen waren (wie Sonntage oder Feiertage) oder nur sehr begrenzt geöffnet hatten.

`Variation zwischen den Filialen`: Während einige Muster über die Filialen hinweg ähnlich sind, gibt es auch Unterschiede in den Verkaufszahlen, was auf verschiedene Standortfaktoren, Filialgrößen oder Kundenbasen hindeuten könnte.

### 3. Beschreiben Sie einige prägnante Zusammenhänge, die Ihnen zwischen verschiedenen Features auffallen.

In [292]:
train_df.groupby('DayOfWeek')['Sales'].mean()

DayOfWeek
1    7901.422431
2    7036.199486
3    6617.223371
4    6369.797242
5    6831.456849
6    6011.092986
7     125.497440
Name: Sales, dtype: float64

In [297]:
train_df.groupby('StoreType')['Sales'].mean()

StoreType
StoreType A     5878.099497
StoreType C     5938.888868
StoreType D     5640.378560
StoreTypee B    9348.329020
Name: Sales, dtype: float64

In [300]:
train_df.groupby('Assortment')['Sales'].mean()

Assortment
Basic       5518.713299
Extended    6161.353333
Extra       9609.298824
Name: Sales, dtype: float64

In [298]:
train_df['Sales'].corr(train_df['CompetitionDistance'])

-0.03567522080901382

In [294]:
train_df.groupby('SchoolHoliday')['Sales'].mean()

SchoolHoliday
0    5704.275184
1    6488.649432
Name: Sales, dtype: float64

In [295]:
train_df.groupby('Promo')['Sales'].mean()

Promo
0    4479.205735
1    8067.441795
Name: Sales, dtype: float64

In [421]:
train_df.corr(numeric_only=True)

,Store ID,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,...,WeekOfYear,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
Store ID,1.000000e+00,-3.564871e-07,0.002610,0.019468,-0.001721,0.000040,NaN,-0.000181,0.000341,0.000842,...,0.000819,-0.015520,0.049279,-0.055231,-0.016331,-0.039262,0.067460,0.003277,0.067364,0.008053
DayOfWeek,-3.564871e-07,1.000000e+00,-0.447948,-0.389245,-0.534218,-0.393895,NaN,-0.211693,0.001060,-0.004684,...,-0.002239,0.000088,-0.000027,-0.000029,0.000034,0.000025,0.000118,0.000168,0.000118,0.000033
Sales,2.609588e-03,-4.479479e-01,1.000000,0.914052,0.659275,0.434767,NaN,0.075003,0.017092,0.043193,...,0.046544,-0.025104,0.080109,-0.035675,0.052101,0.062046,-0.139425,-0.070549,-0.139451,-0.148157
Customers,1.946830e-02,-3.892450e-01,0.914052,1.000000,0.611782,0.316520,NaN,0.066329,-0.001499,0.035291,...,0.038278,-0.166500,0.020418,-0.109836,0.061680,0.088983,-0.189678,-0.121545,-0.189678,-0.188520
Open,-1.721330e-03,-5.342182e-01,0.659275,0.611782,1.000000,0.299531,NaN,0.078737,0.002016,-0.005740,...,-0.001133,-0.001793,0.001943,0.009735,0.007548,0.007890,-0.008748,-0.008479,-0.008750,-0.007646
Promo,3.958783e-05,-3.938946e-01,0.434767,0.316520,0.299531,1.000000,NaN,0.055701,0.025531,-0.013782,...,-0.000294,-0.000623,0.000153,0.000172,-0.000186,-0.000143,-0.000826,-0.001156,-0.000826,-0.000246
StateHoliday,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SchoolHoliday,-1.811101e-04,-2.116930e-01,0.075003,0.066329,0.078737,0.055701,NaN,1.000000,-0.036184,0.101233,...,0.065138,0.006260,-0.002563,-0.001666,-0.001328,-0.003471,-0.006431,-0.007385,-0.006430,-0.005278
Year,3.413860e-04,1.059909e-03,0.017092,-0.001499,0.002016,0.025531,NaN,-0.036184,1.000000,-0.319445,...,-0.309757,-0.005841,0.001466,0.001636,-0.001788,-0.001368,-0.007753,-0.010868,-0.007758,-0.002299
Month,8.419804e-04,-4.683789e-03,0.043193,0.035291,-0.005740,-0.013782,NaN,0.101233,-0.319445,1.000000,...,0.967092,-0.014494,0.003644,0.004063,-0.004443,-0.003399,-0.019238,-0.026971,-0.019251,-0.005703


In [ ]:
exception_days = train_df[(train_df['DayOfWeek'] == 7) & (train_df['Open'] == 1) & (train_df['Sales'] > 0)]
px.line(exception_days, x='Date', y='Sales', color='Store ID')

- An Tagen mit Promotion (Promo = 1) liegt der durchschnittliche Umsatz bei etwa 8.067 €, während er an Tagen ohne Promotion (Promo = 0) bei etwa 4.479 € liegt. 
    - Promotionen hat einen deutlich positiven Einfluss auf den Umsatz haben.
- Der durchschnittliche Umsatz variiert je nach Wochentag, wobei Montag der umsatzstärkste Tag ist (ca. 7.901 €). Der Umsatz nimmt im Laufe der Woche tendenziell ab und erreicht am Sonntag den niedrigsten Wert (ca. 125 €).
    - Wochentag hat einen signifikanten Einfluss auf den Umsatz.
- Der durchschnittliche Umsatz während der Schulferien (SchoolHoliday = 1) liegt bei ca. 6.489 €, was höher ist als außerhalb der Schulferien (ca. 5.704 €). 
    - Dies könnte darauf hindeuten, dass Schulferien zu einem Anstieg des Umsatzes führen.
- Der durchschnittliche Umsatz variiert je nach Filialtyp als auch nach Sortimentskategorie. Filialen vom Typ B erzielen mit durchschnittlich etwa 9.348 € den höchsten Umsatz, gefolgt von Typ C, A und D.
    - Der Filialtyp als auch die Sortimentskategorie hat einen Einfluss auf den Umsatz haben.
- 

# Predictive Modeling

In [509]:
train_df['StoreType'] = train_df['StoreType'].map({"StoreType A": 1, "StoreTypee B": 2, "StoreType C": 3, "StoreType D": 4})
train_df['Assortment'] = train_df['Assortment'].map({"Basic": 0, "Extra": 1, "Extended": 2})
train_df['PromoInterval']= train_df['PromoInterval'].map({0:0, 'Jan,Apr,Jul,Oct' : 1, 'Feb,May,Aug,Nov' : 2, 'Mar,Jun,Sept,Dec' : 3})

train_df['StateHoliday'] = train_df['StateHoliday'].map({"Kein Feiertag": 0, "Feiertag": 1, "Ostern": 2, "Weihnachten": 3})

In [346]:
# train_df.drop(['Promo2SinceWeek', 'Promo2SinceYear', 'StateHoliday'], axis=1, inplace=True)

In [511]:
X = train_df
y = train_df['Sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [513]:
# spalten zum droppen
drop_colmn = ['Sales', 'Customers', 'Date', 'Promo2SinceWeek', 'Promo2SinceYear', 'StateHoliday']

# für StandardScaler
ss_colmn = ['Store ID', 'Year', 'Month', 'Day', 'WeekOfYear', 'CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear']
# ss_colmn = ['Store ID', 'Month', 'Day', 'WeekOfYear', 'Promo2SinceWeek']

# für One Hot Encoding
ohe_colmn = ['StateHoliday', 'StoreType', 'Assortment', 'PromoInterval', 'Season']

In [517]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error, accuracy_score, r2_score
from time import time

In [519]:
models = [
    ('LinearRegression', LinearRegression()),
    ('Ridge', Ridge(random_state=42)),
    ('Lasso', Lasso(random_state=42)),
    ('DecisionTreeRegressor', DecisionTreeRegressor(random_state=42)),
    ('GradientBoostingRegressor', GradientBoostingRegressor(random_state=42)),
    ('RandomForestRegressor', RandomForestRegressor(random_state=42))
]

preprocessor = ColumnTransformer(
    transformers=[
        ('drop_cols', 'drop', drop_colmn),
        ('scale', StandardScaler(), ss_colmn),
        ('onehot', OneHotEncoder(handle_unknown='ignore'), ohe_colmn)
    ], remainder='passthrough')

In [ ]:
results = []

for model_name, model in models:
    # Modellpipeline erstellen
    model_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])

    # Zeitmessung starten
    start_time = time()

    # Modell trainieren
    model_pipeline.fit(X_train, y_train)

    # Zeitmessung beenden
    end_time = time()
    training_time = end_time - start_time

    # Vorhersagen treffen
    y_pred = model_pipeline.predict(X_test)

    # Performance bewerten
    r2_score = model_pipeline.score(X_test, y_test)
    mse = mean_squared_error(y_test, y_pred)

    # Ergebnisse speichern
    results.append({
        'model_name': model_name,
        'r2_score': r2_score,
        'mse': mse,
        'training_time': training_time
    })

# Ergebnisse in DataFrame umwandeln
results_df = pd.DataFrame(results)

In [521]:
results_df

,model_name,r2_score,mse,training_time
0,LinearRegression,0.555377,7.143584e+06,0.504773
1,Ridge,0.555400,7.143215e+06,0.239728
2,Lasso,0.555346,7.144090e+06,1.820407
3,DecisionTreeRegressor,0.930104,1.122990e+06,2.571619
4,GradientBoostingRegressor,0.723370,4.444511e+06,40.676421
5,RandomForestRegressor,0.961455,6.192915e+05,164.631409


## Randomforest

In [514]:
# Vorverarbeitungspipeline erstellen
preprocessor = ColumnTransformer(
    transformers=[
        ('drop_cols', 'drop', drop_colmn),
        ('scale', StandardScaler(), ss_colmn),
        ('onehot', OneHotEncoder(handle_unknown='ignore'), ohe_colmn)
    ], remainder='passthrough')

# Modellpipeline erstellen
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_jobs=-1, random_state=42))
])

# Hyperparameter-Raster definieren
param_grid = {
    'regressor__n_estimators': [300],
    'regressor__max_depth': [None],
    #'regressor__min_samples_split': [2,4,8]
}

# Grid-Suche durchführen
gs = GridSearchCV(model, param_grid=param_grid, cv=3, n_jobs=-1)
gs.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('drop_cols',
                                                                         'drop',
                                                                         ['Sales',
                                                                          'Customers',
                                                                          'Date',
                                                                          'Promo2SinceWeek',
                                                                          'Promo2SinceYear',
                                                                          'StateHoliday']),
                                                                        ('scale',
                                                                         StandardScaler(),
                                                                         ['Store '
                                                                          'ID',
                                                                          'Year',
                                                                          'Month',
                                                                          'Day',
                                                                          'WeekOfYear',
                                                                          'CompetitionDistance',
                                                                          'CompetitionOpenSinceMonth',
                                                                          'CompetitionOpenSinceYear']),
                                                                        ('onehot',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['StateHoliday',
                                                                          'StoreType',
                                                                          'Assortment',
                                                                          'PromoInterval',
                                                                          'Season'])])),
                                       ('regressor',
                                        RandomForestRegressor(n_jobs=-1,
                                                              random_state=42))]),
             n_jobs=-1,
             param_grid={'regressor__max_depth': [None],
                         'regressor__n_estimators': [300]})

In [522]:
# Get feature names after transformations
transformed_feature_names = gs.best_estimator_.named_steps['preprocessor'].get_feature_names_out()

# Get feature importances
importances = gs.best_estimator_.named_steps['regressor'].feature_importances_

# Create DataFrame
feature_importances = pd.DataFrame({
    'feature': transformed_feature_names,
    'importance': importances
})

feature_importances.sort_values(by='importance', ascending=False)

,feature,importance
28,remainder__Open,0.434150
5,scale__CompetitionDistance,0.164220
0,scale__Store ID,0.066692
29,remainder__Promo,0.065170
6,scale__CompetitionOpenSinceMonth,0.043755
27,remainder__DayOfWeek,0.035003
7,scale__CompetitionOpenSinceYear,0.034859
4,scale__WeekOfYear,0.021989
3,scale__Day,0.020099
12,onehot__StoreType_1,0.012889


In [523]:
y_pre=gs.predict(X_test)

In [516]:
print(f"Accuracy Trainingsdaten: {gs.score(X_train, y_train)}")
print(f"Accuracy Testdaten: {gs.score(X_test, y_test)}")

Accuracy Trainingsdaten: 0.9949558676262471
Accuracy Testdaten: 0.961713877142043


In [524]:
# Annahme: y_pred und y_actual sind NumPy-Arrays oder Listen mit den Vorhersagen und tatsächlichen Umsätzen

# Kosten für Überschätzung
overestimate_cost_1 = 150  # Kosten bei Überschätzung um mehr als 3000 EUR
overestimate_cost_2 = 250  # Kosten bei Überschätzung um mehr als 6000 EUR

# Kosten für Unterschätzung
underestimate_cost = 100  # Kosten bei Unterschätzung um mehr als 4000 EUR

# Berechne die Differenzen zwischen Vorhersagen und tatsächlichen Umsätzen
diff = (y_pre - y_test).to_numpy()

# Initialisiere Gesamtkosten für jedes Modell
total_costs_random_forest = 0

# Iteriere durch die Daten und berechne die Gesamtkosten für jedes Modell
for i in range(len(diff)):
    if diff[i] > 6000:
        total_costs_random_forest += overestimate_cost_2
    elif diff[i] > 3000:
        total_costs_random_forest += overestimate_cost_1
    elif diff[i] < -4000:
        total_costs_random_forest += underestimate_cost


# Drucke die Gesamtkosten für jedes Modell

print("Gesamtkosten Random Forest:", total_costs_random_forest)


Gesamtkosten Random Forest: 35300
